In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('rdds') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/04 06:03:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.5'

In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [8]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [14]:
rows = rdd.take(10)
row = rows[0]

In [15]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [9]:
from datetime import datetime

In [10]:
start = datetime(year=2020, month=1, day=1)

In [11]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [12]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [16]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [21]:
from collections import namedtuple

In [22]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [23]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [27]:
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, DoubleType

In [28]:
result_schema = StructType([StructField('hour', TimestampType(), True), 
                            StructField('zone', IntegerType(), True), 
                            StructField('revenue', DoubleType(), True), 
                            StructField('count', IntegerType(), True)])

In [29]:
#rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(1)
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [30]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('revenue', DoubleType(), True), StructField('count', IntegerType(), True)])

In [31]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-15 20:00:00| 260|163.90000000000003|   14|
|2020-01-29 19:00:00| 166| 695.0099999999999|   45|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-04 20:00:00| 129|            583.27|   38|
|2020-01-02 08:00:00|  66|            197.69|   10|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-17 21:00:00| 236|              33.6|    4|
|2020-01-12 12:00:00|  82|            290.41|   14|
|2020-01-28 16:00:00| 197| 831.4399999999998|   18|
|2020-01-10 22:00:00|  95| 407.7100000000002|   37|
|2020-01-10 01:00:00| 215|            109.69|    2|
|2020-01-07 18:00:00|  25| 554.2900000000001|   37|
|2020-01-18 07:00:00|  55|              48.3|    1|
|2020-01-28 09:00:00| 166| 473.0200000000002|   36|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 

In [32]:
df_result.write.parquet('tmp/green-revenue')

In [33]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [34]:
import pandas as pd

In [35]:
rows = duration_rdd.take(10)

In [36]:
df = pd.DataFrame(rows, columns=columns)

In [39]:
list(df.itertuples())

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 11:07:42'), PULocationID=179, DOLocationID=223, trip_distance=0.76),
 Pandas(Index=6, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-16 08:22:29'), PULocationID=41, DOLocationID=237, trip_distance=3.32),
 Panda

In [40]:
list(df.iterrows())

[(0,
  VendorID                                2.0
  lpep_pickup_datetime    2020-01-23 13:10:15
  PULocationID                             74
  DOLocationID                            130
  trip_distance                         12.77
  Name: 0, dtype: object),
 (1,
  VendorID                                NaN
  lpep_pickup_datetime    2020-01-20 15:09:00
  PULocationID                             67
  DOLocationID                             39
  trip_distance                           8.0
  Name: 1, dtype: object),
 (2,
  VendorID                                2.0
  lpep_pickup_datetime    2020-01-15 20:23:41
  PULocationID                            260
  DOLocationID                            157
  trip_distance                          1.27
  Name: 2, dtype: object),
 (3,
  VendorID                                2.0
  lpep_pickup_datetime    2020-01-05 16:32:26
  PULocationID                             82
  DOLocationID                             83
  trip_distance          

In [37]:
def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [38]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [56]:
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, TimestampType, DoubleType
pred_schema = StructType([
    StructField("VendorID", DoubleType(), True),
    StructField("lpep_pickup_datetime", TimestampType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("predicted_duration", DoubleType(), True),
    StructField("Index", IntegerType(), True)
])

In [43]:
duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .take(5)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, predicted_duration=63.849999999999994),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, predicted_duration=40.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, predicted_duration=6.35),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, predicted_duration=6.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, predicted_duration=9.200000000000001)]

In [60]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [61]:
df_predicts.schema

StructType([StructField('VendorID', DoubleType(), True), StructField('lpep_pickup_datetime', StructType([]), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('predicted_duration', DoubleType(), True)])

In [62]:
df_predicts.show()

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0|                  {}|          74|         130|        12.77|63.849999999999994|
|     NaN|                  {}|          67|          39|          8.0|              40.0|
|     2.0|                  {}|         260|         157|         1.27|              6.35|
|     2.0|                  {}|          82|          83|         1.25|              6.25|
|     2.0|                  {}|         166|          42|         1.84| 9.200000000000001|
|     2.0|                  {}|         179|         223|         0.76|               3.8|
|     2.0|                  {}|          41|         237|         3.32|16.599999999999998|
|     2.0|                  {}|          75|         161|         2.21|             11.05|

In [42]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

